In [1]:
#!pip install graphdatascienceb

In [1]:
# Define connection details
uri = "bolt://localhost:7687"  # Replace with your Neo4j URI
username = "username"         # Replace with your Neo4j username
password = "password"      # Replace with your Neo4j password
db_name = "nutrition"          # Specify the database you want to work on

In [2]:
from graphdatascience import GraphDataScience
import pandas as pd

# Create another GraphDataScience instance and set the default database
gds = GraphDataScience(uri, auth=(username, password), database=db_name)

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('advertisedListenAddress' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated field from a procedure. ('serverLocation' returned by 'gds.debug.arrow' is deprecated.)} {position: line: 1, column: 1, offset: 0} for query: 'CALL gds.debug.arrow()'


2.7.0


### Check ingredients to ensure they are in the knowledge graph

In [21]:
cypher_query = """
CALL apoc.load.json("file:///survey_data_version_4.json") YIELD value
WITH 
    value.user.likes_ingredients AS likesIngredients,
    value.user.dislikes_ingredients AS dislikesIngredients,
    value.user.allergic_to_ingredients AS allergicToIngredients

OPTIONAL MATCH (existingIngredient:Ingredient)
WITH likesIngredients, dislikesIngredients, allergicToIngredients, COLLECT(existingIngredient.name) AS existingIngredients

WITH 
    [ingredient IN likesIngredients WHERE NOT ingredient IN existingIngredients] AS newLikesIngredients, 
    dislikesIngredients, allergicToIngredients, existingIngredients
WITH 
    newLikesIngredients,
    [ingredient IN dislikesIngredients WHERE NOT ingredient IN existingIngredients] AS newDislikesIngredients,
    allergicToIngredients, existingIngredients
WITH 
    newLikesIngredients, 
    newDislikesIngredients, 
    [ingredient IN allergicToIngredients WHERE NOT ingredient IN existingIngredients] AS newAllergicToIngredients, 
    existingIngredients
RETURN 
    newLikesIngredients AS missingLikesIngredients,
    newDislikesIngredients AS missingDislikesIngredients,
    newAllergicToIngredients AS missingAllergicToIngredients
"""

# Run the Cypher query
result = gds.run_cypher(cypher_query)

# Print the result

result

,missingLikesIngredients,missingDislikesIngredients,missingAllergicToIngredients
0,[],[],[]
1,[],[],[]
2,[],[intestine],[]
3,[],[],[]
4,[],[],[]
5,[],[],[]
6,[octopus],[durian],[]
7,[],[],[high caffein]
8,[whey protein],[],[]
9,[durian],[],[]


In [14]:
'''
MATCH (i:Ingredient) 
WHERE apoc.text.levenshteinSimilarity(i.name, 'intestine') > 0.5
RETURN i.name, apoc.text.levenshteinSimilarity(i.name, 'intestine') AS similarity
ORDER BY similarity DESC
'''

"\nMATCH (i:Ingredient) \nWHERE apoc.text.levenshteinSimilarity(i.name, 'intestine') > 0.5\nRETURN i.name, apoc.text.levenshteinSimilarity(i.name, 'intestine') AS similarity\nORDER BY similarity DESC\n"